In Notebook 02A, we saw that our linear models seemd to perform atleast as well as our non-linear models. Here, we formally evaluate this. We use sklearn's default hyperparameters on each model type. 

In [1]:
# import torch
import argparse
import joblib
from pathlib import Path
import os

from tqdm import tqdm

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import Lasso,Ridge,ElasticNet
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.model_selection import GridSearchCV
import xgboost as XGB

import plotnine as p9
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
random_state = 42
seed = random_state

data_path = '/nobackup/users/hmbaghda/metastatic_potential/'

cv_folds = 10

n_cores = 30
os.environ["OMP_NUM_THREADS"] = str(n_cores)
os.environ["MKL_NUM_THREADS"] = str(n_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(n_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(n_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(n_cores)

In [18]:
model_types = {'KNN': KNN(n_jobs = int(n_cores/cv_folds)),
               'RF': RandomForestRegressor(n_jobs = int(n_cores/cv_folds), random_state = random_state),
               'poly_SVR': SVR(kernel = 'poly'), 
               'rbf_SVR': SVR(kernel = 'rbf'),
               'XGB': XGB.XGBRegressor(n_jobs = int(n_cores/cv_folds), random_state = random_state), 
               'linear_SVR': SVR(kernel = 'linear'), 
               'Ridge': Ridge(random_state = random_state), 
               'Lasso': Lasso(random_state = random_state),
               'PLSR': PLSRegression(scale=False),
              }

In [19]:
X = pd.read_csv(os.path.join(data_path, 'processed',  'expr.csv'), index_col = 0).values
y = pd.read_csv(os.path.join(data_path, 'processed', 'metastatic_potential.csv'), index_col = 0)['mean'].values.ravel()